In [1]:
!pip install -q transformers[torch]
!pip install -q datasets
!pip install -q huggingface_hub
!pip install -q accelerate -U
!pip install -q bitsandbytes
!pip install -q peft
!pip install -q trl
!pip install -q huggingface_hub
!pip install -q openai
!pip install -q matplotlib
!pip install -q evaluate[evaluator]
!pip install -q rouge_score
!pip install -q bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 6.8 MB/s eta

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, GenerationConfig, pipeline, Trainer, DataCollatorWithPadding
from trl import SFTTrainer
from peft import get_peft_model, PeftConfig, PeftModel, LoraConfig, prepare_model_for_kbit_training
from datasets import Dataset, load_dataset
import evaluate
import torch
import huggingface_hub
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import csv
import json
from math import sqrt
from openai import OpenAI

#### Dataset

In [11]:
!git clone https://github.com/wyim/aci-bench

fatal: destination path 'aci-bench' already exists and is not an empty directory.


In [12]:
aci_bench_files = {"train": "aci-bench/data/challenge_data/train.csv", "valid": "aci-bench/data/challenge_data/valid.csv", "test":  "aci-bench/data/challenge_data/clinicalnlp_taskB_test1.csv"}
aci_bench_dataset = load_dataset("csv", data_files=aci_bench_files)

print(aci_bench_dataset)

DatasetDict({
    train: Dataset({
        features: ['dataset', 'encounter_id', 'dialogue', 'note'],
        num_rows: 67
    })
    valid: Dataset({
        features: ['dataset', 'encounter_id', 'dialogue', 'note'],
        num_rows: 20
    })
    test: Dataset({
        features: ['dataset', 'encounter_id', 'dialogue', 'note'],
        num_rows: 40
    })
})


#### Testing

In [13]:
client = OpenAI(api_key="")

In [14]:
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")
bertscore = evaluate.load("bertscore")

In [21]:
example = aci_bench_dataset["train"][0]["note"]


def generate_summary_gpt_3(conversation):
  response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
      {"role": "system", "content": "You are a medical assistant that writes accurate clinical notes from doctor-patient conversations."},
      {"role": "user", "content": f"Write a summary of the following conversation between a and a patient. The summary should be similar in format to the following: {example}.\nHere is the conversation to summarise:\n{conversation}"},
    ]
  )

  return response.choices[0].message.content

In [22]:
def generate_summary_finetuned_gpt_3(conversation):
  response = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-1106:personal::8uExj74y",
    messages=[
      {"role": "system", "content": "You are a medical assistant that writes accurate clinical notes from doctor-patient conversations"},
      {"role": "user", "content": f"Write a clinical report from the following doctor-patient conversation: {conversation}"},
    ]
  )

  return response.choices[0].message.content

In [23]:
gpt_predictions = []
gpt_ground_truth = []

for element in aci_bench_dataset["test"]:
    summary = generate_summary_gpt_3(element["dialogue"])
    gpt_predictions.append(summary)
    gpt_ground_truth.append(element["note"])

In [31]:
gpt_rouge_results = rouge.compute(predictions=gpt_predictions, references=gpt_ground_truth)
gpt_bleu_results = bleu.compute(predictions=gpt_predictions, references=gpt_ground_truth)
gpt_bertscore_results = bertscore.compute(predictions=gpt_predictions, references=gpt_ground_truth, lang="en")

print("--- Regular - Rouge ---\n", json.dumps(gpt_rouge_results, indent=4))
print("--- Regular - Bleu ---\n", json.dumps(gpt_bleu_results, indent=4))

print("Regular precision: ", np.mean(gpt_bertscore_results["precision"]))
print("Regular precision STDDEV: ", np.std(gpt_bertscore_results["precision"]))

print("Regular recall: ", np.mean(gpt_bertscore_results["recall"]))
print("Regular recall STDDEV: ", np.std(gpt_bertscore_results["recall"]))

print("Regular f1: ", np.mean(gpt_bertscore_results["f1"]))
print("Regular f1 STDDEV: ", np.std(gpt_bertscore_results["f1"]))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


--- Regular - Rouge ---
 {
    "rouge1": 0.2187807540986606,
    "rouge2": 0.08276789811335514,
    "rougeL": 0.1394140348868187,
    "rougeLsum": 0.20421743156268596
}
--- Regular - Bleu ---
 {
    "bleu": 0.0017490650737455604,
    "precisions": [
        0.5578925872983459,
        0.2050440844781628,
        0.09439983528927322,
        0.05313210667769278
    ],
    "brevity_penalty": 0.011301196874216705,
    "length_ratio": 0.18238700906907018,
    "translation_length": 9794,
    "reference_length": 53699
}
Regular precision:  0.8115471646189689
Regular precision STDDEV:  0.012963193903091776
Regular recall:  0.7685287773609162
Regular recall STDDEV:  0.0108125426207668
Regular f1:  0.7893723025918007
Regular f1 STDDEV:  0.008805002555590292


In [32]:
gpt_fine_tuned_predictions = []
gpt_fine_tuned_ground_truth = []

for element in aci_bench_dataset["test"]:
    summary = generate_summary_finetuned_gpt_3(element["dialogue"])
    gpt_fine_tuned_predictions.append(summary)
    gpt_fine_tuned_ground_truth.append(element["note"])

In [33]:
gpt_fine_tuned_rouge_results = rouge.compute(predictions=gpt_fine_tuned_predictions, references=gpt_fine_tuned_ground_truth)
gpt_fine_tuned_bleu_results = bleu.compute(predictions=gpt_fine_tuned_predictions, references=gpt_fine_tuned_ground_truth)
gpt_fine_tuned_bertscore_results = bertscore.compute(predictions=gpt_fine_tuned_predictions, references=gpt_fine_tuned_ground_truth, lang="en")

print("--- Fine-Tuned - Rouge ---\n", json.dumps(gpt_fine_tuned_rouge_results, indent=4))
print("--- Fine-Tuned - Bleu ---\n", json.dumps(gpt_fine_tuned_bleu_results, indent=4))

print("Fine-tuned precision: ", np.mean(gpt_fine_tuned_bertscore_results["precision"]))
print("Fine-tuned precision STDDEV: ", np.std(gpt_fine_tuned_bertscore_results["precision"]))

print("Fine-tuned recall: ", np.mean(gpt_fine_tuned_bertscore_results["recall"]))
print("Fine-tuned recall STDDEV: ", np.std(gpt_fine_tuned_bertscore_results["recall"]))

print("Fine-tuned f1: ", np.mean(gpt_fine_tuned_bertscore_results["f1"]))
print("Fine-tuned f1 STDDEV: ", np.std(gpt_fine_tuned_bertscore_results["f1"]))

--- Fine-Tuned - Rouge ---
 {
    "rouge1": 0.3168599441175324,
    "rouge2": 0.09902986451977086,
    "rougeL": 0.1581909258636383,
    "rougeLsum": 0.295782761848691
}
--- Fine-Tuned - Bleu ---
 {
    "bleu": 0.02327410698550368,
    "precisions": [
        0.4714627151051625,
        0.1428639846743295,
        0.06309980806142035,
        0.03639423076923077
    ],
    "brevity_penalty": 0.20869658985476425,
    "length_ratio": 0.3895789493286653,
    "translation_length": 20920,
    "reference_length": 53699
}
Fine-tuned precision:  0.8061729043722152
Fine-tuned precision STDDEV:  0.013953139663691223
Fine-tuned recall:  0.7796061620116234
Fine-tuned recall STDDEV:  0.013918131011446212
Fine-tuned f1:  0.7925805985927582
Fine-tuned f1 STDDEV:  0.011245254804112727


#### Evaluation

In [72]:
!git clone https://github.com/nazmulkazi/dataset_automated_medical_transcription.git

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Cloning into 'dataset_automated_medical_transcription'...
remote: Enumerating objects: 578, done.
remote: Counting objects: 100% (578/578), done.
remote: Compressing objects: 100% (351/351), done.
remote: Total 578 (delta 233), reused 567 (delta 227), pack-reused 0
Receiving objects: 100% (578/578), 5.65 MiB | 14.87 MiB/s, done.
Resolving deltas: 100% (233/233), done.


In [73]:
transcripts_path = "./dataset_automated_medical_transcription/transcripts/transcribed/"
casenotes_path = "./dataset_automated_medical_transcription/casenotes/annotator_1/"

In [79]:
def prepare_transcripts(path):
    files = os.listdir(path)
    result = []

    for file in files:
        with open(path + file, "r") as raw_data:
            data = json.load(raw_data)

            conversation = ""
            for turn in data:
                speaker = "Doctor" if turn["speaker"] == 1 else "Patient"
                dialogue = " ".join(turn["dialogue"])
                conversation += f"{speaker}: {dialogue}\n"

            result.append(conversation)

    return result

def prepare_casenotes(path):
    files = os.listdir(path)
    annotators = os.listdir("./dataset_automated_medical_transcription/casenotes/")
    result = []

    categories = ["Client Details", "Chief Complaint", "History of Present Illness", "Past Psychiatric History", "History of Substance Use", "Social History", "Family History", "Review of Systems"]

    # Iterating over annotator #1
    for file in files:
        with open(path + file, "r") as raw_data:
            data = json.load(raw_data)

            casenotes = {}
            for category in categories:
                casenotes[category] = []

            for element in data:
                category = categories[int(element["categoryId"])]
                casenotes[category].append(element["formalText"])

            # Iterating over other annotators to find casenotes for the same case
            for annotator in annotators:
                if annotator != "annotator_1":
                    annotations = os.listdir("./dataset_automated_medical_transcription/casenotes/" + annotator)
                    if file in annotations:
                        with open("./dataset_automated_medical_transcription/casenotes/" + annotator + "/" + file, "r") as raw_data:
                            data = json.load(raw_data)

                            for element in data:
                                category = categories[int(element["categoryId"])]
                                if len(casenotes[category]) == 0:
                                    casenotes[category].append(element["formalText"])

            casenote = ""
            for category in casenotes.keys():
                casenote += f"{category}:\n"

                content = casenotes[category] if len(casenotes[category]) > 0 else ["None"]

                casenote += " ".join(content)
                casenote += "\n\n"
            result.append(casenote)

    return result

transcripts = prepare_transcripts(transcripts_path)
case_notes = prepare_casenotes(casenotes_path)

psy_dataset = {"transcripts": transcripts, "case_notes": case_notes}
psy_dataset = Dataset.from_dict(psy_dataset)
print(psy_dataset)

Dataset({
    features: ['transcripts', 'case_notes'],
    num_rows: 71
})


In [85]:
test_set = psy_dataset.select(range(30))

In [86]:
gpt_predictions = []
gpt_ground_truth = []

for element in test_set:
    summary = generate_summary_gpt_3(element["transcripts"])
    gpt_predictions.append(summary)
    gpt_ground_truth.append(element["case_notes"])

In [87]:
gpt_rouge_results = rouge.compute(predictions=gpt_predictions, references=gpt_ground_truth)
gpt_bleu_results = bleu.compute(predictions=gpt_predictions, references=gpt_ground_truth)
gpt_bertscore_results = bertscore.compute(predictions=gpt_predictions, references=gpt_ground_truth, lang="en")

print("--- Regular - Rouge ---\n", json.dumps(gpt_rouge_results, indent=4))
print("--- Regular - Bleu ---\n", json.dumps(gpt_bleu_results, indent=4))

print("Regular precision: ", np.mean(gpt_bertscore_results["precision"]))
print("Regular precision STDDEV: ", np.std(gpt_bertscore_results["precision"]))

print("Regular recall: ", np.mean(gpt_bertscore_results["recall"]))
print("Regular recall STDDEV: ", np.std(gpt_bertscore_results["recall"]))

print("Regular f1: ", np.mean(gpt_bertscore_results["f1"]))
print("Regular f1 STDDEV: ", np.std(gpt_bertscore_results["f1"]))

--- Regular - Rouge ---
 {
    "rouge1": 0.21523492922225795,
    "rouge2": 0.036223608128307425,
    "rougeL": 0.10709606659184955,
    "rougeLsum": 0.18857667344669893
}
--- Regular - Bleu ---
 {
    "bleu": 0.0038235126430255263,
    "precisions": [
        0.3330785191656656,
        0.04459296592527665,
        0.005496317467296911,
        0.00132348075438403
    ],
    "brevity_penalty": 0.2108928833803408,
    "length_ratio": 0.3911743346576103,
    "translation_length": 9157,
    "reference_length": 23409
}
Regular precision:  0.8086453835169475
Regular precision STDDEV:  0.017470944564082685
Regular recall:  0.814163879553477
Regular recall STDDEV:  0.015240598551046059
Regular f1:  0.8113403658072154
Regular f1 STDDEV:  0.014970191192520452


In [88]:
gpt_fine_tuned_predictions = []
gpt_fine_tuned_ground_truth = []

for element in test_set:
    summary = generate_summary_finetuned_gpt_3(element["transcripts"])
    gpt_fine_tuned_predictions.append(summary)
    gpt_fine_tuned_ground_truth.append(element["case_notes"])

In [89]:
gpt_fine_tuned_rouge_results = rouge.compute(predictions=gpt_fine_tuned_predictions, references=gpt_fine_tuned_ground_truth)
gpt_fine_tuned_bleu_results = bleu.compute(predictions=gpt_fine_tuned_predictions, references=gpt_fine_tuned_ground_truth)
gpt_fine_tuned_bertscore_results = bertscore.compute(predictions=gpt_fine_tuned_predictions, references=gpt_fine_tuned_ground_truth, lang="en")

print("--- Fine-Tuned - Rouge ---\n", json.dumps(gpt_fine_tuned_rouge_results, indent=4))
print("--- Fine-Tuned - Bleu ---\n", json.dumps(gpt_fine_tuned_bleu_results, indent=4))

print("Fine-tuned precision: ", np.mean(gpt_fine_tuned_bertscore_results["precision"]))
print("Fine-tuned precision STDDEV: ", np.std(gpt_fine_tuned_bertscore_results["precision"]))

print("Fine-tuned recall: ", np.mean(gpt_fine_tuned_bertscore_results["recall"]))
print("Fine-tuned recall STDDEV: ", np.std(gpt_fine_tuned_bertscore_results["recall"]))

print("Fine-tuned f1: ", np.mean(gpt_fine_tuned_bertscore_results["f1"]))
print("Fine-tuned f1 STDDEV: ", np.std(gpt_fine_tuned_bertscore_results["f1"]))

--- Fine-Tuned - Rouge ---
 {
    "rouge1": 0.2896863587212526,
    "rouge2": 0.06237394797763583,
    "rougeL": 0.12634566844392298,
    "rougeLsum": 0.2471446495923787
}
--- Fine-Tuned - Bleu ---
 {
    "bleu": 0.02461088788758558,
    "precisions": [
        0.3609808642757572,
        0.08183087861858812,
        0.01749141330619514,
        0.004906958959979607
    ],
    "brevity_penalty": 0.6167619906331893,
    "length_ratio": 0.6741851424665727,
    "translation_length": 15782,
    "reference_length": 23409
}
Fine-tuned precision:  0.8184127668539684
Fine-tuned precision STDDEV:  0.007840054240356187
Fine-tuned recall:  0.8263047218322754
Fine-tuned recall STDDEV:  0.013454719233891535
Fine-tuned f1:  0.8222729583581289
Fine-tuned f1 STDDEV:  0.00812910794507994


In [34]:
classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [61]:
def extract_patient_dialogue(data):
    patient_dialogue = []
    split_data = data.split("\n")
    for line in split_data:
        # if "[patient]" in line:
        if "Patient:" in line:
            patient_dialogue.append(line)
    return "\n".join(patient_dialogue)

In [62]:
def compute_emotions(text):
  combined_scores = {}

  text_length = len(text)
  chunks = 0

  for start in range(0, text_length, 512):
    end = min(text_length, start+512)
    chunk = text[start: end]
    chunks += 1

    emotions = classifier(chunk)[0]

  for emotion in emotions:
    label = emotion["label"]

    if label not in combined_scores.keys():
      combined_scores[label] = emotion["score"]
    else:
      combined_scores[label] += emotion["score"]

  return [{'label': label, 'score': score / chunks} for label, score in combined_scores.items()]

In [69]:
def calculate_distance(list1, list2):
  if len(list1) != len(list2):
    raise ValueError("Lists must have the same length.")

  distance = 0

  for i in range(len(list1)):
    dict1 = list1[i]
    dict2 = list2[i]

    if set(dict1.keys()) != set(dict2.keys()):
      raise ValueError("Dictionaries must have the same labels.")

    for label in dict1.keys():
      if label != 'label':
        distance += abs(dict1[label] - dict2[label])

  return distance

In [70]:
distances = []

for i in range(len(gpt_predictions)):
  dialogue = aci_bench_dataset["test"][i]["dialogue"]
  patient_dialogue = extract_patient_dialogue(dialogue)

  if (len(patient_dialogue) > 0):    
      patient_emotions = compute_emotions(patient_dialogue)
      clinical_note_emotions = compute_emotions(gpt_predictions[i])
    
      distance = calculate_distance(patient_emotions, clinical_note_emotions)
      distances.append(distance)

print("Average distance - Regular: ", np.mean(distances))
print("STDDEV - Regular: ", np.std(distances))

Average distance - Regular:  0.2010427877300287
STDDEV - Regular:  0.10589316448806478


In [71]:
fine_tuned_distances = []

for i in range(len(gpt_fine_tuned_predictions)):
  dialogue = aci_bench_dataset["test"][i]["dialogue"]
  patient_dialogue = extract_patient_dialogue(dialogue)

  if (len(patient_dialogue) > 0):    
      patient_emotions = compute_emotions(patient_dialogue)
      clinical_note_emotions = compute_emotions(gpt_fine_tuned_predictions[i])
    
      distance = calculate_distance(patient_emotions, clinical_note_emotions)
      fine_tuned_distances.append(distance)

print("Average distance - Fine-tuned: ", np.mean(fine_tuned_distances))
print("STDDEV - Fine-tuned: ", np.std(fine_tuned_distances))

Average distance - Fine-tuned:  0.1955050075653339
STDDEV - Fine-tuned:  0.10056420155081257
